# 加州房价分析与预测

### 项目简介：
#### 该项目的数据集基于1990年美国加州地区人口普查的数据，数据集记录了加州地区所有以街区为单位的每个区域的人口数量、经纬度、房子总数、收入中位数、房屋价值中位数等10个属性的数据。本项目主要以房屋价值中位数为标签，其余属性为特征，通过使用python编程，利用pyhon相关库，如numpy、pandas、matplotlib等数据处理、分析和可视化库，结合sklearn中相关机器学习模型接口，对加州地区的数据进行探索分析、预处理、特征选择、建模分析、模型评估等完整的机器学习过程，以加深和巩固自己在数据分析和机器学习自学过程中的所学。

### 0.导入相关库

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

### 1.读取数据，并进行数据探索

In [ ]:
data_path='/kaggle/input/california-housing-prices/housing.csv'

使用pandas的read_csv()函数读取csv格式的数据集，保存格式为DataFrame。读取数据时保留原始数据备份，另行复制一份以免出现难以挽回的操作失误导致数据丢失。

In [ ]:
data_init = pd.read_csv(data_path)
data = data_init.copy()

data.head()函数读取数据前5行，data.tail()读取尾部默认5行数据。也可以读取指定行数的数据，将行数传入函数即可,如data.head(10)。

In [ ]:
data.head()

data.info()函数可以查看数据集的总体情况：样本数/特征数（含标签）/特征缺失/特征的数据类型/数据集大小

In [ ]:
#可以看出名为’total_bedrooms‘的属性部分值有缺失，后续将考虑填充。此外除'ocean_proximity'为对象类型外，其余属性均为浮点型数值类型。
data.info()

data.describe()函数只针对数值型数据操作，data可以是DataFrame/Series。主要展示数值型数据的部分统计特性，如均值/标准差/最大最小值/上下四分位数等。也可以指定特定百分比的数值，如5%，10%等处的值，此处保持默认参数。

In [ ]:
data.describe()

#### 本项目主要研究加州的房价预测问题，所以将房价有关的'median_house_value'属性设置为标签，其余属性为特征

In [ ]:
feature=data.drop('median_house_value',axis=1)
label=data['median_house_value']

对标签数据进行探索分析

In [ ]:
# 房价中位数在500001.0处的统计最多，共计3842个唯一值,占到总数据长度的18.6%
label.value_counts()

In [ ]:
len(label.unique())/len(label)

对标签使用describe()函数查看统计特性

In [ ]:
label.describe()

绘制标签的直方图，查看数据分布

In [ ]:
label.hist(bins=50,figsize=(8,6),color='b',alpha=.7)

plt.title('label')
plt.xlabel('house_value')
plt.ylabel('counts')
plt.grid(False)      #不显示网格
plt.show()

对特征数据进行探索分析----绘制数值型属型特征的直方图，展示数据分布情况

In [ ]:
feature.hist(bins=50,figsize=(20,15),color='b',alpha=.7)
plt.show()

除数值型数据外，特征集还有一个数据类型为'object'的特征。

如果只有一个数据类型为'object'的特征，我们可以直接选择它。如果特征数量太多，就不容易从太多特征中刻意找出某种类型的众多特征，所以此处遍历数据的columns属性，再筛选出符合dtype==‘object’的特征。

In [ ]:
#筛选出数据类型为'object'的特征
category_list=[column for column in feature.columns if feature[column].dtype=='object']
category_list
feature[category_list]

In [ ]:
#统计该类别特征的值的分布，因为dataframe格式无法使用value_counts()方法，所以将该特征用series格式呈现
feature.loc[:,category_list[0]].value_counts()

文本类型的数据虽然可用于决策树、随机森林等不对特征类型做要求的机器学习算法，但是不转换成数值型数据就无法使用专职于数值型数据的学习算法，譬如线性回归、逻辑回归、K近邻、支持向量机、神经网络等算法。

此步骤为数据探索阶段，暂不对文本数据进行数值型数据的转换操作，后续数据处理阶段再进行。

#### 相关性探索
使用corr()方法很容易计算出每对属性之间的标准相关系数（又称皮尔逊相关系数）,data.corr()输出为dataframe格式,每一列为某个属性与其余属性的相关系数，当然此处也只针对数值型属性计算相关性。

In [ ]:
corr = data.corr()
corr

可以单独查看与标签的相关性，可以看出median_income与之相关性较高，这也很好理解：收入高，购买的房价大概率也高。

可根据特征与标签的相关性强度，对特征进行选择。特征选择除相关性方法外，还有方差过滤法（发散程度越小，特征价值越小）、卡方检验、F检验（分类）、t检验（回归）、互信息、封装法、嵌入法等，此处不作详细介绍。

In [ ]:
#只考虑相关性大小，不考虑正负，并按照绝对值大小排序
corr['median_house_value'].abs().sort_values(ascending=False)

除了考虑现有的特征以外，我们还可以生成新的特征，例如'total_bedrooms'、'population'与标签的相关性都不高，我们可以组合一个新的特征‘bedrooms_per_population’ 由 total_bedrooms/population产生，再看新特征与标签的相关性

In [ ]:
#结果显示，bedrooms_per_population的相关性要比total_bedrooms和population各自与标签的相关性都要高
#而bedrooms_per_house的相关性就没有total_bedrooms和households各自与标签的相关性都要高
data_1=data.copy()
data_1['bedrooms_per_population']=data_1['total_bedrooms']/data_1['population']
data_1['bedrooms_per_house']=data_1['total_bedrooms']/data_1['households']
data_1.corr()['median_house_value'].abs().sort_values(ascending=False)

基于上述新特征相较于老特征的高相关性，我们可将'bedrooms_per_population'加入数据集。因total_rooms还存在空值，所以将在后续步骤构建新征'bedrooms_per_population'。

#### 地理数据可视化
特征集有出现经纬度两个属性，可以考虑绘制地理可视化，每个样本其实代表一个类似街道的区域，所以经纬度的散点图可以表征这些街道的地理信息，同时可以选择使用人口特征来表征街道的规模，使用房屋中位数的价格来表征该街道房屋的价值。

In [ ]:
data.plot(kind='scatter',x='longitude',y='latitude'
          ,s=data['population']/100,label='population'#以人口密度值区别散点的大小
          ,alpha=.4                                 #设置小的透明度，会突出颜色更深的点
          ,figsize=(16,12)                           #设置画布大小
          ,c='median_house_value'                   #颜色深度以房价高低衡量
          ,cmap=plt.get_cmap('jet')                 #选择colormap
          ,colorbar=True
         ) 
plt.legend()
plt.show()

#### 以上几步数据探索可以借用pandas_profiling库实现，一行代码搞定数据概览、变量分析、相关性分析等操作

In [ ]:
import pandas_profiling #数据探索分析库，简单高效生成交互式数据报告
data_profile=data.profile_report(style={'full_width':True})
data_profile

### 2.数据预处理 

#### 数据集划分，创建训练集和测试集
在将数据喂给机器学习模型前，我们会首先将数据集划分为训练集和测试集，训练集用于训练模型，测试集用于检测模型的性能（分类或回归预测的好坏）。根据测试集模型的性能表现，再对模型加以改进。或选择其他模型、或选择新的特征、亦或者调整超参数，后续再说。

#### 自定义函数用于划分训练集和测试集

In [ ]:
def split_train_test(data,test_ratio):
    np.random.seed(42)
    shuffled_index = np.random.permutation(len(data))#随机生成指定长度范围内不重复随机数序列
    test_set_size = int(len(data)*test_ratio)
    test_index = shuffled_index[:test_set_size]
    train_index = shuffled_index[test_set_size:]
    return data.iloc[train_index],data.iloc[test_index]

In [ ]:
test_ratio=0.2
train_set,test_set=split_train_test(data,test_ratio)
print(len(train_set),'train_set',len(test_set),'test_set')

#### 使用sklearn中的train_test_split函数划分训练集和数据集
可以直接对包含标签的数据集划分为训练集和测试集

In [ ]:
from sklearn.model_selection import train_test_split
train_f,test_f = train_test_split(data,test_size=0.2,random_state=42)#添加随机种子保证每次运行划分的都是一样的结果

也可以对分离的特征集和标签集划分为训练集的特征和标签、测试集的特征和标签

In [ ]:
train_x,test_x,train_y,test_y= train_test_split(feature,label,test_size=0.2,random_state=42)

#### 分层抽样划分训练集和测试集 （根据相关程度较高的特征作为分层抽样依据）
分层抽样可以让训练集合测试集在某个重要特征上表现出相似的数据分布。相关矩阵显示，‘median_house_value’与房价的相关性较高，因此选择‘median_house_value’作为分层抽样的划分依据

In [ ]:
corr_matrix = data.corr()
corr_matrix['median_house_value'].abs().sort_values(ascending=False)

由于收入中位数是连续值，需将其离散化创建一个收入类别特征

median_income均值3.87，标准差1.9，所以收入中位数有68%的样本落入在2-6万区间（将其近似看作正太分布），与下图的直方图比较吻合

In [ ]:
data['median_income'].describe()

In [ ]:
plt.figure()
plt.hist(data['median_income'],bins=50)
# plt.grid()
plt.show()

In [ ]:
data['income_cat'] = np.ceil(data['median_income']/1.5)
data['income_cat'].where(data['income_cat']<5,5.0,inplace=True)# series.where(),小于5则保持原样，大于5赋值为5.0
data['income_cat'].value_counts()

In [ ]:
plt.figure()
plt.hist(data['income_cat'])
# plt.grid()
plt.show()

### 使用sklearn中的StratifiedShuffleSplit进行分层抽样

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1       #进行一次划分
                               ,test_size=0.2    #测试集占比0.2
                               ,random_state=42  #设置随机种子保证每次运行划分的数据集不会发生变化
                              )
for train_index,test_index in split.split(data
                                          ,data['income_cat']#选择分层抽样依据的属性（或特征）
                                         ):
    strat_train_set=data.loc[train_index]
    strat_test_set=data.loc[test_index]

#### 不同抽样方法的数据分布比较

#### 分层抽样的‘income_cat’分布

In [ ]:
x_strat=strat_train_set['income_cat'].value_counts(normalize=True)
x_strat

#### 全分布数据集中‘income_cat’分布

In [ ]:
x_full=data['income_cat'].value_counts(normalize=True)
x_full

#### 使用train_test_split随机划分的数据集‘income_cat’分布

In [ ]:
train_rand,test_rand=train_test_split(data,test_size=0.2,random_state=42)
x_rand=train_rand['income_cat'].value_counts(normalize=True)
x_rand

对全样本、分层抽样、随机抽样三种数据分布进行比较，发现分层抽样可以和全样本在数据的统计分布上相差较小，而随机抽样的数据分布与全样本的分布偏差相较于分层抽样的偏差要大。

In [ ]:
strat_bia=(x_strat-x_full)/x_full*100
rand_bia=(x_rand-x_full)/x_full*100
compare=pd.DataFrame([x_full,x_strat,x_rand,strat_bia,rand_bia],index=['x_full','x_strat','x_rand','strat_bia_%','rand_bia_%']).T
compare

创建的收入类比特征主要用于分层抽样，如果不保留，可以删除

In [ ]:
for set in (strat_train_set,strat_test_set):
    set.drop(['income_cat'],axis=1)
    

### 机器学习算法的数据准备阶段

#### 数据清理 :清理缺失值（清理特征或样本）/填充缺失值（0,均值，中位数，机器学习模型填充等）

In [ ]:
data1=data.drop('income_cat',axis=1)

In [ ]:
data1.isnull().sum()

几种简单数据清理的方法，此处我们选择中位数填充法

In [ ]:
# hdata.drop(['total_bedrooms'],axis=1,inplace=True)
#data.dropna(subset=['total_bedrooms'])
data_median_fill=data1.fillna(data1.median())
data_median_fill.info()

#### 使用sklearn中的SimpleImputer,对缺失值统一填充,只能对数值型数据操作

In [ ]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='median')#以中位数填充缺失值
data_num=data.drop('ocean_proximity',axis=1)
columns=data_num.columns
data_num=imputer.fit_transform(data_num)
data_num=pd.DataFrame(data_num,columns=columns)


#### 新特征的产生

还记得之前的新特征'bedrooms_per_population'吗？之前因为没有处理total_bedrooms中的缺失值就没添加新特征。此步我们将其纳入数据集中

In [ ]:
data_median_fill['bedrooms_per_population']=data_median_fill['total_bedrooms']/data_median_fill['population']

#### 到此，虽然我们还没对数值型特征进行标准化，对文本类特征进行数字编码等操作，但是我们已经可以把数据喂给机器学习模型(暂时先不将文本类型输入模型中)，如决策数、随机森林等。

In [ ]:
#分层抽样,这里我们将先前分层抽样的几步抽象成一个函数，传入要分层抽样的数据集和分层抽样的特征，返回训练集和测试集
def stratifiedshufflesplit(data_,feature='median_income'):
    data_['cat'] = np.ceil(data_[feature]/1.5)           #此处我们默认使用收入中位数作为分层抽样的依据
    data_['cat'].where(data_['cat']<5,5.0,inplace=True)

    from sklearn.model_selection import StratifiedShuffleSplit
    split_ = StratifiedShuffleSplit(n_splits=1   #进行一次划分
                               ,test_size=0.2    #测试集占比0.2
                               ,random_state=42  #设置随机种子保证每次运行划分的数据集不会发生变化
                              )
    for train_index,test_index in split.split(data_
                                          ,data_['cat']#选择分层抽样依据的属性（或特征）
                                         ):
        strat_train=data_.loc[train_index]
        strat_test=data_.loc[test_index]
        strat_train=strat_train.drop('cat',axis=1)
        strat_test=strat_test.drop('cat',axis=1)
    data_.drop('cat',axis=1,inplace=True)
    return strat_train,strat_test

strat_train,strat_test=stratifiedshufflesplit(data_median_fill,feature='median_income')

train_x,train_y=strat_train.drop(['median_house_value','ocean_proximity'],axis=1),strat_train['median_house_value']
test_x,test_y=strat_test.drop(['median_house_value','ocean_proximity'],axis=1),strat_test['median_house_value']

决策树模型

回归模型的评判标准，此处用均方根误差来衡量。均方根误差越小，说明预测值与实际值越接近。

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr=DecisionTreeRegressor(max_depth=10)
dtr=dtr.fit(train_x,train_y)

from sklearn.metrics import mean_squared_error
predict=dtr.predict(test_x)
mse=mean_squared_error(test_y,predict)
rmse1=np.sqrt(mse)
rmse1

随机森林回归模型

随机森立的结果明显好于决策树

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfg=RandomForestRegressor(n_estimators=10,max_depth=10,random_state=0)
rfg=rfg.fit(train_x,train_y)

predicts=rfg.predict(test_x)
mse_=mean_squared_error(test_y,predicts)
rmse_2=np.sqrt(mse_)
rmse_2

#### 考虑将数值型特征标准化或归一化，以及文本类型数据进行数值化编码操作
先将数据集中的数值型特征和文本类型特征区分开

In [ ]:
data2=data_median_fill.copy()
data2_num=data2.drop(['ocean_proximity','median_house_value'],axis=1)
data2_cat=data2['ocean_proximity']
data2_label=data2['median_house_value']

使用OneHotEncoder对文本数据进行编码。注意传入实例的数据必须是二维数据，单独的series需转换成二维数组或DataFrame格式

In [ ]:
from sklearn.preprocessing import OneHotEncoder
data2_cat=pd.DataFrame(data2_cat)

encoder=OneHotEncoder(categories='auto')
data2_cat_onehot=encoder.fit_transform(data2_cat).toarray()    #输出是稀疏矩阵的一种存储方式，需转换成数组

#独热编码实例的学习参数，显示多个文本类型数据在编码前的文本值数组，每一个文本特征存储一个数组
encoder.categories_

In [ ]:
data2_cat_onehot=pd.DataFrame(data2_cat_onehot,columns=encoder.categories_)
data2_cat_onehot.head()

数值型特征的列名

In [ ]:
columns_list=data2_num.columns.tolist()

独热编码后的文本类特征列名加入到数值特征列名的列表中，表示全部特征的列名

In [ ]:
columns_list.extend(encoder.categories_[0].tolist())
columns_list

#### 数值型特征归一化
归一化：Y=(X-Y.min())/(Y.max()-Y.min())

In [ ]:
data2_num = data2_num.sub(data2_num.min())/(data2_num.max()-data2_num.min())

使用describe()可以看出所有数值型特征均进行了归一化，数值范围[0,1]

In [ ]:
data2_num.describe()

In [ ]:
data2_num_onehot=pd.DataFrame(np.c_[data2_num,data2_cat_onehot],columns=columns_list)#np.c_[]数组横向拼接成数组,np.r_[]纵向拼接
data2_num_onehot

In [ ]:
data2=pd.concat([data2_num_onehot,data2_label],axis=1)
data2.info()

使用分层抽样划分训练集合测试集

In [ ]:
strat_train,strat_test=stratifiedshufflesplit(data2,feature='median_income')

train_x,train_y=strat_train.drop(['median_house_value'],axis=1),strat_train['median_house_value']
test_x,test_y=strat_test.drop(['median_house_value'],axis=1),strat_test['median_house_value']

归一化和独热编码前后决策树表现分别为

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr=DecisionTreeRegressor(max_depth=10)
dtr=dtr.fit(train_x,train_y)

from sklearn.metrics import mean_squared_error
predict=dtr.predict(test_x)
mse=mean_squared_error(test_y,predict)
rmse3=np.sqrt(mse)
print('独热编码前后决策树表现分别为：\n前：{}，\n后：{}'.format(rmse1,rmse3))

模型的学习参数dtr.feature_importances_显示特征重要性,所有特征重要性之和为1

In [ ]:
dtr.feature_importances_

重要特征及其重要百分比

In [ ]:
feature_sort_dtr=list(zip(dtr.feature_importances_,train_x.columns))#重要性在前，排序按照元祖对第一个元素排序
sorted(feature_sort_dtr,reverse=True)

In [ ]:
corr['median_house_value'].abs().sort_values(ascending=False)

最好的特征是median_income,这与数据探索分析阶段的相关性的重要程度不谋而合

In [ ]:
train_x.columns[np.argmax(dtr.feature_importances_)]

归一化和独热编码前后随机森林表现分别为

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfg=RandomForestRegressor(n_estimators=10,max_depth=10)
rfg=rfg.fit(train_x,train_y)

predicts=rfg.predict(test_x)
mse_=mean_squared_error(test_y,predicts)
rmse_4=np.sqrt(mse_)
print('独热编码前后随机森林表现分别为：\n前：{}，\n后：{}'.format(rmse_2,rmse_4))

In [ ]:
feature_sort_rfg=list(zip(rfg.feature_importances_,train_x.columns))
sorted(feature_sort_rfg,reverse=True)

In [ ]:
train_x.columns[np.argmax(rfg.feature_importances_)]

#### 保存模型 
使用python的pickel模块或者sklearn.externals.joblib

In [ ]:
# pip install joblib 直接安装joblib,无需从sklearn.externals模块导入

In [ ]:
# from sklearn.externals import joblib
import joblib
joblib.dump(dtr,'dtr.pkl')
joblib.dump(rfg,'rfg.pkl')

保存的模型和之前的dtr模型参数一致

In [ ]:
dtr_load=joblib.load('dtr.pkl')
dtr_load

In [ ]:
dtr

模型的装载

In [ ]:
rfg_load=joblib.load('rfg.pkl')
rfg_load

#### 参数调整
网格搜索法寻找最佳参数组合

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid=dict(n_estimators=[10,30],max_depth=[4,6,8,10],min_samples_split=[2,3,4],min_samples_leaf=[2,3,4])
rfg_gs=RandomForestRegressor()
grid_search=GridSearchCV(rfg_gs,param_grid,cv=5,scoring='neg_mean_squared_error')

grid_search.fit(train_x,train_y)

网格搜索属性best_params_，显示最佳学习参数

In [ ]:
grid_search.best_params_

最好的估算器

In [ ]:
grid_search.best_estimator_

网格搜索每一个参数的结果

In [ ]:
grid_search.cv_results_

保存最好的模型估算器

In [ ]:
best_model=grid_search.best_estimator_
joblib.dump(best_model,'best_model_rfg.pkl')

In [ ]:
test_predictions=best_model.predict(test_x)
test_mse=mean_squared_error(test_y,test_predictions)
test_rmse=np.sqrt(test_mse)

In [ ]:
test_rmse

In [ ]:
best_model.feature_importances_

In [ ]:
best_model.get_params()

In [ ]:
best_model.set_params()

对估算器的个数进行细化调整，在30附近扩大范围

In [ ]:
param_grid=dict(n_estimators=list(range(20,41)),max_depth=[10],min_samples_split=[3],min_samples_leaf=[4])
rfg_gs_estimator=RandomForestRegressor()
grid_search=GridSearchCV(rfg_gs_estimator,param_grid,cv=5,scoring='neg_mean_squared_error')

grid_search.fit(train_x,train_y)

In [ ]:
test_predictions=grid_search.predict(test_x)
test_mse=mean_squared_error(test_y,test_predictions)
test_rmse=np.sqrt(test_mse)
test_rmse

In [ ]:
grid_search.best_params_

In [ ]:
best_model_1=grid_search.best_estimator_
joblib.dump(best_model_1,'best_model_1.pkl')